# Data Preprocessing - Pipeline Template

Template fürs Data Preprorocessing nach dem Feedback vom Coaching am 27.04.2023:
1. Daten einlesen
2. Daten vorbereiten
3. Schritte der Vorbereitung auswerten mithilfe einer Linearen Regression


## 1. Daten einlesen

In [1]:
import pandas as pd
import numpy as np
from scipy import stats
import sys
import os
from dotenv import load_dotenv

In [2]:
#add working directory to sys path to execute utils/dataset.py
working_dir = os.environ.get("WORKING_DIRECTORY")
sys.path.insert(0, working_dir)

from utils.dataset import get_data 

df = get_data()
df = df.dropna()
df.head()

Loading data from wines: 8000it [00:00, 25100.32it/s]


,wine type,fixed acidity,volatile acidity,citric acid,residual sugar,magnesium,flavanoids,minerals,calcium,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,Pinot noir,5.8,0.15,0.49,1.1,76.729301,894.94,186.639301,109.91,0.048,21.0,98.0,0.99290,3.19,0.48,9.2,5
1,Merlot,6.6,0.25,0.32,5.6,4.795712,1160.95,251.875712,247.08,0.039,15.0,68.0,0.99163,2.96,0.52,11.1,6
2,Chardonnay,6.7,0.21,0.34,1.5,85.193710,789.82,304.703710,219.51,0.035,45.0,123.0,0.98949,3.24,0.36,12.6,7
3,Merlot,8.3,0.28,0.27,17.5,11.976525,777.86,237.586525,225.61,0.045,48.0,253.0,1.00014,3.02,0.56,9.1,6
4,Merlot,7.5,0.42,0.19,6.9,5.599673,785.72,95.399673,89.80,0.041,62.0,150.0,0.99508,3.23,0.37,10.0,6


In [3]:
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, FunctionTransformer
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC

In [4]:
X_train, X_test, y_train, y_test = train_test_split(df.drop("quality", 1),
													df["quality"],
													test_size = 0.2, random_state = 0)


/var/folders/qf/_mzt5nz96y12mx5szzg62j480000gn/T/ipykernel_44758/4197502663.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  X_train, X_test, y_train, y_test = train_test_split(df.drop("quality", 1),


In [5]:
trf1 = ColumnTransformer(transformers =[
	('cat', SimpleImputer(strategy ='most_frequent'), ["wine type"]),
	('num', SimpleImputer(strategy ='median'), ["alcohol", "sulphates", "pH", "volatile acidity", "citric acid", "residual sugar", "chlorides", "density", "total sulfur dioxide", "free sulfur dioxide", "fixed acidity"]),
	
], remainder ='passthrough')


In [6]:
first_step = trf1.fit_transform(X_train)
pd.DataFrame(first_step).head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,Chardonnay,9.1,0.45,3.06,0.28,0.54,12.9,0.049,0.9984,152.0,62.0,7.3,4.99912,1258.06,280.71912,275.72
1,Merlot,8.8,0.54,3.05,0.23,0.45,10.6,0.094,0.99699,169.0,49.0,6.1,62.088643,927.24,378.028643,315.94
2,Cabernet Sauvignon,12.9,0.34,3.27,0.29,0.54,5.7,0.035,0.98976,155.0,54.0,5.0,34.740787,949.66,300.610787,265.87
3,Merlot,12.9,0.62,3.11,0.64,0.28,4.4,0.032,0.99036,78.0,19.0,6.6,0.301944,766.31,243.931944,243.63
4,Cabernet Sauvignon,11.0,0.32,3.04,0.23,0.36,7.0,0.036,0.9938,161.0,43.0,7.5,92.369473,846.55,378.239473,285.87


In [7]:
trf2 = ColumnTransformer(transformers =[
    ('enc', OneHotEncoder(sparse = False, drop ='first'), list(range(1))),
], remainder ='passthrough')

In [8]:
second_step = trf2.fit_transform(first_step)
pd.DataFrame(second_step).head()

/opt/anaconda3/envs/data-science/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
0,1.0,0.0,0.0,0.0,9.1,0.45,3.06,0.28,0.54,12.9,0.049,0.9984,152.0,62.0,7.3,4.99912,1258.06,280.71912,275.72
1,0.0,0.0,1.0,0.0,8.8,0.54,3.05,0.23,0.45,10.6,0.094,0.99699,169.0,49.0,6.1,62.088643,927.24,378.028643,315.94
2,0.0,0.0,0.0,0.0,12.9,0.34,3.27,0.29,0.54,5.7,0.035,0.98976,155.0,54.0,5.0,34.740787,949.66,300.610787,265.87
3,0.0,0.0,1.0,0.0,12.9,0.62,3.11,0.64,0.28,4.4,0.032,0.99036,78.0,19.0,6.6,0.301944,766.31,243.931944,243.63
4,0.0,0.0,0.0,0.0,11.0,0.32,3.04,0.23,0.36,7.0,0.036,0.9938,161.0,43.0,7.5,92.369473,846.55,378.239473,285.87


In [9]:
pipe = Pipeline(steps =[
    ('tf1', trf1),
    ('tf2', trf2),
    ("svc", SVC())
])

In [10]:
test = pipe.fit(X_train, y_train)
pipe.score(X_test, y_test)

ValueError: could not convert string to float: 'Chardonnay'